## VectorStoreRetriever란?
- VectorStore를 Retriever 인터페이스로 변환하는 가벼운 래퍼 클래스
- Vector Store의 검색 메서드(similarity search, MMR 등)를 사용하여 문서를 검색
- Runnable 인터페이스를 구현하여 LCEL 체인에 쉽게 통합 가능

### 핵심 개념
- **VectorStore**: 벡터 유사도를 기반으로 문서를 저장하고 검색하는 데이터베이스
- **Retriever**: 문서 검색을 위한 표준 인터페이스를 제공하는 추상화
- **LCEL 체인**: VectorStoreRetriever는 Runnable이므로 다른 구성요소와 체인으로 연결 가능

### VectorStoreRetriever의 장점
- **표준화된 인터페이스**: 다양한 Vector Store를 동일한 방식으로 사용 가능
- **체인 통합**: `invoke()`, `batch()` 등의 표준 메서드 지원
- **유연한 검색**: 다양한 검색 유형과 파라미터 지원


In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

## TextLoader를 사용하여 test.txt 파일 읽어오기

TextLoader는 LangChain에서 제공하는 기본적인 텍스트 파일 로더입니다.
- 텍스트 파일을 Document 객체로 변환
- 파일 경로를 메타데이터로 자동 저장
- 인코딩 설정 가능 (기본값: utf-8)


In [ ]:
from langchain_community.document_loaders import TextLoader

# test.txt 파일의 경로 설정
file_path = "../data/ai.txt"

# TextLoader를 사용하여 텍스트 파일 로드
loader = TextLoader(file_path, encoding='utf-8')

# 문서 로드
document = loader.load()

**문서 나누기**

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],  # 문단 → 줄 → 문장 → 단어 순
    chunk_size=250,    
    chunk_overlap=30
)

chunks = splitter.split_documents(document)

print(f"총 {len(chunks)}개 청크 생성\n")

임베딩 객체 생성

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

Vector Store 생성

In [ ]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents=chunks, embedding=embeddings)

### 검색 유형별 설정

**1. 유사도 검색 (기본값)**

In [ ]:
query = "딥러닝의 주요 아키텍처와 그 활용 분야를 설명해줘"

In [ ]:
# 유사도 검색
similarity_retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 검색 문서 수를 3개로 설정
)

print(f"질문 : {query}")
results = similarity_retriever.invoke(query)
print("=== 유사도 검색 (k=3) ===")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content}")


**MMR (Maximum Marginal Relevance) 검색**  
- 검색된 문서의 다양성과 관련성 간의 균형을 맞추기 위해 사용하는 검색 전략  
- 특히 유사한 결과가 중복되는 것을 방지하고, **사용자의 쿼리와 관련 있으면서도 서로 다른 정보를 제공**받고 싶을 때 사용합니다.

- 이 문서가 질문과 관련이 있으면서, 동시에 지금까지 뽑아놓은 문서들과 얼마나 다른 새로운 정보를 담고 있는가? 를 고려한다고 생각하면됨

| 매개변수 이름              | 설명                                                   |
| -------------------- | ---------------------------------------------------- |
| `search_type`        | `"mmr"`로 설정하면 MMR 검색 방식 사용. (기본은 `"similarity"`)     |
| `k`                  | 최종적으로 반환할 문서 수 (예: 5이면 5개 문서 반환)                     |
| `fetch_k`            | 후보 문서 수. `k`개를 고르기 위해 먼저 몇 개를 가져올지 (기본값은 보통 20\~100) |
| `lambda_mult`        | λ (람다) 값. 관련성과 다양성 사이의 균형 (0\~1 사이의 값)               |
| `relevance_score_fn` | 유사도를 계산하는 함수. 기본은 cosine similarity, 필요하면 사용자 정의 가능  |
| `filter`             | 검색 전 문서 필터링 조건 (예: 메타데이터 필터링)                        |


In [ ]:
# 2. MMR Search  
mmr_retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3,  # 검색 문서 수를 3개로 설정
        "lambda_mult": 0.7,  # 다양성 중시
    }
)

print(f"질문 : {query}")
mmr_results = mmr_retriever.invoke(query)
print("=== MMR 검색 (k=3) ===")
for i, doc in enumerate(mmr_results):
    print(f"{i+1}. {doc.page_content}")
